In [1]:
#모듈 임포트
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.model_selection import KFold
#이미지 사이즈
size = 28
h = os.getenv('HOME')+'/aiffel/'
file_name = 'rock_scissor_paper_model.h5'

In [2]:
#모델 생성
def make_your_models():
    model=keras.models.Sequential()
    model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(size,size,3)))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    return model

In [3]:
# 이미지 리사이즈
def resize_img(img_path):
    for path in img_path:
        images = glob.glob(path + "/*.jpg")
        target_size = (size,size)
        for img in images:
            old_img = Image.open(img)
            new_img = old_img.resize(target_size,Image.ANTIALIAS)
            new_img.save(img,"JPEG")
    print("all of images are resized")

In [4]:
# 리사이즈 된 이미지를 행렬화
def load_data(path,number_of_data=6012,f = 'jpg'):
    # 바위 : 0 가위 : 1 보 : 2
    img_size = size
    color = 3
    total = number_of_data*img_size*img_size*color
    imgs = np.zeros(total,dtype = np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype = np.int32)
    idx = 0
    
    for file in glob.iglob(path+'/rock/*.'+ f):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img
        labels[idx] = 0
        idx+=1
        
    for file in glob.iglob(path+'/scissor/*.'+ f):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img
        labels[idx] = 1
        idx+=1
        
    for file in glob.iglob(path+'/paper/*.'+ f):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:] = img
        labels[idx] = 2
        idx+=1
    
    return imgs,labels

In [5]:
#이미지 리사이즈
p = os.getenv('HOME')+'/aiffel/rock_scissor_paper/'
image_dir_path = [p+'scissor',p+'rock',p+'paper']
resize_img(image_dir_path)

all of images are resized


In [6]:
#교차 검증을 위해 trian data에서 test셋을 나눈다 
kf = KFold(n_splits = 5,shuffle = True)

In [7]:
images_dir_path = os.getenv('HOME')+'/aiffel/rock_scissor_paper'
(x_train,y_train) = load_data(images_dir_path)
#이미지 정규화
x_train_norm = x_train/255.0
x_train_reshaped = x_train_norm.reshape(-1,size,size,3)

In [8]:
#교차검증 모델 학습
model = make_your_models()
for train_idx, test_idx in kf.split(x_train_reshaped):
    X_train,X_test = x_train_reshaped[train_idx,:,:,:],x_train_reshaped[test_idx,:,:,:]
    Y_train,Y_test = y_train[train_idx],y_train[test_idx]
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy'
            ,metrics=['accuracy'])
    model.fit(x_train_reshaped,y_train,epochs = 5,validation_data=(X_test,Y_test))
    score = model.evaluate(X_test,Y_test)
    print("test_loss: {} ".format(score[0]))
    print("test_accuracy: {}".format(score[1]))

Epoch 1/5
188/188 [==============================] - 4s 5ms/step - loss: 0.8104 - accuracy: 0.6164 - val_loss: 0.3749 - val_accuracy: 0.8512
Epoch 2/5
188/188 [==============================] - 1s 3ms/step - loss: 0.2676 - accuracy: 0.9004 - val_loss: 0.2098 - val_accuracy: 0.9285
Epoch 3/5
188/188 [==============================] - 1s 3ms/step - loss: 0.1446 - accuracy: 0.9536 - val_loss: 0.1027 - val_accuracy: 0.9684
Epoch 4/5
188/188 [==============================] - 1s 4ms/step - loss: 0.0814 - accuracy: 0.9759 - val_loss: 0.0618 - val_accuracy: 0.9809
Epoch 5/5
38/38 [==============================] - 0s 2ms/step - loss: 0.0360 - accuracy: 0.9909
test_loss: 0.03596573695540428 
test_accuracy: 0.9908561706542969
Epoch 1/5
188/188 [==============================] - 1s 4ms/step - loss: 0.0415 - accuracy: 0.9859 - val_loss: 0.0464 - val_accuracy: 0.9850
Epoch 2/5
188/188 [==============================] - 1s 4ms/step - loss: 0.0318 - accuracy: 0.9887 - val_loss: 0.0184 - val_accuracy

In [9]:
#모델 파일로 저장
model.save(h+file_name)
#파일로 된 모델 호출
my_model = keras.models.load_model(h+file_name)

In [11]:
#테스트 데이터 생성
t_path = os.getenv('HOME')+'/aiffel/test/'
pp = [t_path+'paper',t_path+'rock',t_path+'scissor']
resize_img(pp)

all of images are resized


In [12]:
#이미지 변환 및 정규화
(x_test,y_test) = load_data(t_path,300)
x_test_norm = x_test/255.0
x_test_reshaped = x_test_norm.reshape(-1,size,size,3)

In [13]:
test_loss,test_accuracy = my_model.evaluate(x_test_reshaped,y_test,verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 2.1770 - accuracy: 0.6233
test_loss: 2.1769943237304688 
test_accuracy: 0.6233333349227905


In [ ]:
predicted_result = model.predict(x_test_reshaped)  # model이 추론한 확률값. 
predicted_labels = np.argmax(predicted_result, axis=1)
import random
wrong_predict_list=[]
for i, _ in enumerate(predicted_labels):
    # i번째 test_labels과 y_test이 다른 경우만 모아 봅시다. 
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)

# wrong_predict_list 에서 랜덤하게 5개만 뽑아봅시다.
samples = random.choices(population=wrong_predict_list, k=5)

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    plt.imshow(x_test[n], cmap=plt.cm.binary)
    plt.show()

# 회고합시다

1. 어려웠던 점
    * 테스트 정확도  
        테스트 정확도가 생각보다 매우 낮게 나와 조금 정신이 없었다.  
        
2. 알아낸 점
    1. 정확도가 낮았던 이유  
        앞서 정확도가 기준에 비해서도 너무 낮게 나왔었는데 결론은 training data가 너무 적었기 때문이었다.  제공된 training data가 300장이었고 숫자처럼 비교적 간단한 이미지가 아니라 판단 근거가 적은 상태로 적은양의 데이터로만 학습을 반복해 학습 데이터에서만 작동이 잘되는 overfitting상태에 빠진것으로 보였다.  
          
    2. 정확도를 올리기위해 시도할만한 것들  
        1. 학습 데이터의 양을 늘린다.
        2. 교차검증을 사용한다.

3. 평가 지표를 맞추기 위해 시도한 것들
    1. 학습데이터  
        우선 학습 데이터를 대폭 늘려 사용했다. 
        아무래도 이미지에 배경도 있고 사람마다 손 모양도 다른 복잡한 이미지이다보니  
        일단 많은 이미지를 학습시켰다.  
        </br>
    2. 교차검증
        과적합상태에 대한 다른 해결책으로 교차검증을 사용했다.  
        교차검증은 학습 데이터의 일부를 테스트 데이터로 사용하는 것을 반복해  
        학습의 효율을 늘리는 방법이다.
    